# Cuaderno 13: Combinando Series y DataFrames

A menudo la información requerida para un análisis de datos proviene de múltiples fuentes y está en diversos formatos. En estos casos es necesario *combinar* la información de diferentes objetos `Series` y `DataFrames`. Esta tarea puede ser tan sencilla como concatenar un conjunto de valores con otro, o tan compleja como realizar cruces de tablas al estilo de lo que ocurre en una base de datos.

Examinaremos en este cuaderno dos maneras para combinar `Series` y `DataFrames`: a través de su concatenación con la función `concat` y el método `append`; y a través de las operaciones de cruce de datos sofisticadas implementadas en la función `merge` y el método `join`. Empezamos por importar los módulos de `pandas`y `numpy`:

In [1]:
# importar pandas y NumPy
import numpy as np
import pandas as pd

## Combinando  con `concat` y `append`

Vamos a importar tres DataFrames con el ranking de las universidades de Ecuador, Colombia y Perú obtenido del sitio web de [webometrics](http://www.webometrics.info/es). La información está contenida en las hojas `Ecuador`, `Perú` y `Colombia` del archivo `webometrics.xlsx`:

In [ ]:
# ranking de universidades según webometrics
dfEcuador=pd.read_excel('webometrics.xlsx', sheet_name='Ecuador', index_col='World Rank')
dfColombia=pd.read_excel('webometrics.xlsx', sheet_name='Colombia', index_col='World Rank')
dfPeru=pd.read_excel('webometrics.xlsx', sheet_name='Perú', index_col='World Rank')
display(dfEcuador.head())
display(dfColombia.head())
display(dfPeru.head())


Extraemos la columna `University` de cada `DataFrame` para formar tres series con los nombres de las universidades de cada uno de los países, indexadas por sus posiciones en el ranking mundial.

In [ ]:
sEcuador = dfEcuador['University'] 
sColombia = dfColombia['University'] 
sPeru = dfPeru['University'] 
print(sEcuador)
print('---')
print(sColombia)
print('---')
print(sPeru)

La función `concat` nos permite concatenar las tres series en una serie nueva:

In [ ]:
sjuntas = pd.concat([sEcuador, sPeru, sColombia])
print(sjuntas)

Ordenando esta serie podemos crear un ranking conjunto de las universidades de los tres países:

In [ ]:
sjuntas= sjuntas.sort_index()
print(sjuntas.head(20))

La función `concat` también puede aplicarse a DataFrames:

In [ ]:
dfjuntos = pd.concat([dfEcuador, dfColombia, dfPeru]).sort_index()
dfjuntos.head(20)

En el ejemplo anterior, los índices entre los tres DataFrames no se repetían entre sí. Este no siempre es el caso, y es importante especificar cuál es el tratamiento que debe darse a los índices repetidos al concatenar series o DataFrames.

Supongamos que en lugar del ranking mundial, cada uno de los tres DataFrames estuviera indexado por el ranking de la universidad dentro del país:

In [ ]:
# cambiar el índice de "World Rank" a "ranking"
# notar que es necesario usar reset_index para no perder la columna "World Rank"
dfEcuador.reset_index(inplace=True)
dfEcuador.set_index('ranking', inplace=True)
dfColombia.reset_index(inplace=True)
dfColombia.set_index('ranking', inplace=True)
dfPeru.reset_index(inplace=True)
dfPeru.set_index('ranking', inplace=True)
display(dfEcuador.head())
display(dfColombia.head())
display(dfPeru.head())

Por defecto, `concat` preserva los índices, lo que significa que el nuevo DataFrame tendrá índices duplicados:

In [ ]:
display(pd.concat([dfEcuador, dfColombia, dfPeru]).sort_index().head(20))

Si se fija el parámetro `verify_integrity` al valor de `True`, Pandas indicará un error (excepción `ValueError`) en caso de que el resultado de la combinación tenga índices repetidos:

In [ ]:
display(pd.concat([dfEcuador, dfColombia, dfPeru], verify_integrity= True).sort_index().head(20))

Por el contrario, si el parámetro `ignore_index` se fija a `True`, Pandas desechará el índice de las series o DataFrames que está combinando y creará un índice nuevo:

In [ ]:
display(pd.concat([dfEcuador, dfColombia, dfPeru], ignore_index= True).sort_index().head(20))

Finalmente, puede asignarse al parámetro `keys` una lista con nombres para los DataFrames (o series) que se están combinando. En este caso, `concat`creará un multi-índice donde el nivel superior es el nombre del DataFrame (o de la serie), y el nivel inferior es el índice original:

In [ ]:
dfcombinado = pd.concat([dfEcuador, dfColombia, dfPeru], keys= ['Ecuador', 'Colombia', 'Perú']).sort_index()
idx = pd.IndexSlice
display(dfcombinado.loc[idx[:,0:5],:])

La función `concat` alinea los distintos DataFrames de acuerdo al nombre de sus columnas (es decir, de acuerdo a los valores de los índices de columnas). Si las índices de columnas tienen valores distintos en los DataFrames a combinar, se crea un DataFrame con la *unión* de los índices de columnas y se completan los valores faltantes con `NaN`. Esta operación se conoce como un *outer join*:

In [ ]:
# cambiar el nombre de las tres últimas columnas de dfColombia
dfColombia.rename(columns={'Impact Rank':'impacto', 'Openness Rank' : 'apertura', 
                           'Excellence Rank' : 'excelencia'}, inplace= True)
display(dfColombia)
display(dfEcuador)
display(pd.concat([dfEcuador, dfColombia]))

Si se especifica el valor de `inner` para el parámetro `join`, entonces `concat` retorna un DataFrame donde las columnas están indexadas por la *intersección* de los índices de los DataFrames combinados:

In [ ]:
display(pd.concat([dfEcuador, dfColombia], join='inner'))

Es posible también combinar DataFrames juntando sus columnas en lugar de juntar sus filas. Para ello, debe asignarse el valor de `1` al parámetro `axis`.

Para este ejemplo, trabajaremos con dos DataFrames con la información del número de matrimonios y divorcios en las distintas provincias del país, en el período 2006-2019. La información fue obtenida del [INEC: Instituto Nacional de estadística y censo](https://www.ecuadorencifras.gob.ec/matrimonios-divorcios/).

In [ ]:
dfmatri = pd.read_excel('bdd_Mat-Div.xlsx',sheet_name='Matrimonios')
display(dfmatri.head(5))

dfdivor = pd.read_excel('bdd_Mat-Div.xlsx',sheet_name='Divorcios')
display(dfdivor.head(5))

display(pd.concat([dfmatri, dfdivor], axis=1).head(5))

Empleando el parámetro `keys` podemos crear un multi-índice para las columnas en el DataFrame resultante:

In [ ]:
display(pd.concat([dfmatri, dfdivor], axis=1, keys=['Matrimonios', 'Divorcios']).head(5))

De manera alternativa a la función `concat` puede usarse el método `append`, disponible tanto para series como para DataFrames. Este método crea un nuevo DataFrame y copia los datos de la unión de los DataFrames originales. Por motivos de eficiencia, no se recomienda llamar el método varias veces seguidas para combinar más de dos DataFrames. 

In [ ]:
dfmat_div = dfmatri.append(dfdivor)
display(dfmat_div)

## Combinando con `merge` y `join`

Las operaciones `merge` y `join` son una las características esenciales de Pandas y permiten combinar conjuntos de datos (series o DataFrames) empleando las mismas técnicas eficientes disponibles en los motores bases de datos. Hay tres tipos de combinaciones: *uno-a-uno*, *uno-a-muchos* y *muchos-a-muchos*.

### Combinaciones uno-a-uno

Es la forma más simple de usar la operación `merge`, y se asemeja a la combinación de DataFrames empleando `concat` con la opción `axis=1`:

In [2]:
# importar datos de matrimonios
dfmatri = pd.read_excel('bdd_Mat-Div.xlsx',sheet_name='Matrimonios')
# eliminar región y restringir solamente a los años 2017-2019
dfmatri = dfmatri[['Provincias', 2017, 2018, 2019]]
# renombrar columnas
dfmatri.rename(columns = {2017 : 'M2017', 2018 : 'M2018', 2019 : 'M2019'}, inplace= True)
display(dfmatri.head(5))

# importar datos de divorcios
dfdivor = pd.read_excel('bdd_Mat-Div.xlsx',sheet_name='Divorcios')
# eliminar región y restringir solamente a los años 2017-2019
dfdivor = dfdivor[['Provincias', 2017, 2018, 2019]]
# renombrar columnas
dfdivor.rename(columns = {2017 : 'D2017', 2018 : 'D2018', 2019 : 'D2019'}, inplace= True)
display(dfdivor.head(5))

# combinar con merge
display(pd.merge(dfmatri, dfdivor).head(5))

,Provincias,M2017,M2018,M2019
0,Azuay,3371,3502,3369
1,Bolívar,460,529,499
2,Cañar,1038,1114,1090
3,Carchi,527,446,444
4,Cotopaxi,1805,1839,1794


,Provincias,D2017,D2018,D2019
0,Azuay,2279,2138,1988
1,Bolívar,322,315,299
2,Cañar,641,637,576
3,Carchi,376,381,323
4,Cotopaxi,522,578,602


,Provincias,M2017,M2018,M2019,D2017,D2018,D2019
0,Azuay,3371,3502,3369,2279,2138,1988
1,Bolívar,460,529,499,322,315,299
2,Cañar,1038,1114,1090,641,637,576
3,Carchi,527,446,444,376,381,323
4,Cotopaxi,1805,1839,1794,522,578,602


### Combinaciones uno-a-muchos

En este caso, cada entrada de la columna a fusionar es única en uno de los DataFrames, pero se repite varias veces en el otro DataFrame. 

Suponer, por ejemplo, que un DataFrame contiene una lista de universidades con las ciudades de sus sedes principales, y el otro DataFrame contiene las provincias a las que pertenece cada ciudad:

In [3]:
# DataFrame con universidades y sedes
dfuniversidades = pd.DataFrame({'Universidad' : ['Escuela Politécnica Nacional', 
                                                 'Escuela Politécnica del Litoral', 'Universidad Central', 
                                                'Universidad de Guayaquil', 'Universidad del Azuay'],
                               'Ciudad' : ['Quito', 'Guayaquil', 'Quito', 'Guayaquil', 'Cuenca']})
display(dfuniversidades)

# DataFrame con ciudades y sus provincias
dfprovincias = pd.DataFrame({'Ciudad' : ['Quito', 'Guayaquil', 'Cuenca'],
                             'Provincia' : ['Pichincha', 'Guayas', 'Azuay']})
display(dfprovincias)

#display(pd.merge(dfuniversidades, dfprovincias))
display(pd.merge(dfprovincias, dfuniversidades))


,Universidad,Ciudad
0,Escuela Politécnica Nacional,Quito
1,Escuela Politécnica del Litoral,Guayaquil
2,Universidad Central,Quito
3,Universidad de Guayaquil,Guayaquil
4,Universidad del Azuay,Cuenca


,Ciudad,Provincia
0,Quito,Pichincha
1,Guayaquil,Guayas
2,Cuenca,Azuay


,Ciudad,Provincia,Universidad
0,Quito,Pichincha,Escuela Politécnica Nacional
1,Quito,Pichincha,Universidad Central
2,Guayaquil,Guayas,Escuela Politécnica del Litoral
3,Guayaquil,Guayas,Universidad de Guayaquil
4,Cuenca,Azuay,Universidad del Azuay


### Combinaciones muchos-a-muchos

En este caso, la columna a fusionar tiene elementos repetidos en ambos DataFrames.  

Suponer que un DataFrame contiene el número de paralelos (grupos) que se ofertan de cada materia en un semestre determinado, mientras que un segundo DataFrame indica las materias correspondientes a cada carrera.

In [4]:
# DataFrame con paralelos por materia
dfparalelos = pd.DataFrame({'Materia' : ['Cálculo', 
                                                 'Cálculo', 'Álgebra Lineal', 
                                                'Álgebra Lineal', 'Programación'],
                               'Paralelo' : ['GR1', 'GR2', 'GR1', 'GR2', 'GR1']})
display(dfparalelos)

# DataFrame con ciudades y sus provincias
dfmaterias = pd.DataFrame({'Materia' : ['Cálculo', 'Cálculo', 'Cálculo', 'Álgebra Lineal', 
                                         'Álgebra Lineal', 'Programación', 'Programación'],
                            'Carrera' : ['Matemática', 'Física', 'Economía', 'Matemática ', 
                                         'Física', 'Economía', 'Matemática']})
display(dfmaterias)


,Materia,Paralelo
0,Cálculo,GR1
1,Cálculo,GR2
2,Álgebra Lineal,GR1
3,Álgebra Lineal,GR2
4,Programación,GR1


,Materia,Carrera
0,Cálculo,Matemática
1,Cálculo,Física
2,Cálculo,Economía
3,Álgebra Lineal,Matemática
4,Álgebra Lineal,Física
5,Programación,Economía
6,Programación,Matemática


Al combinar los DataFrames fusionando la columna `Materia`, obtenemos una lista de todos los paralelos ofertados en cada carrera:

In [5]:
display(pd.merge(dfparalelos, dfmaterias))

,Materia,Paralelo,Carrera
0,Cálculo,GR1,Matemática
1,Cálculo,GR1,Física
2,Cálculo,GR1,Economía
3,Cálculo,GR2,Matemática
4,Cálculo,GR2,Física
5,Cálculo,GR2,Economía
6,Álgebra Lineal,GR1,Matemática
7,Álgebra Lineal,GR1,Física
8,Álgebra Lineal,GR2,Matemática
9,Álgebra Lineal,GR2,Física


### Especificando la columna para la combinación

Por defecto, `merge` combina los dos DataFrames por todas las columnas que tengan el mismo nombre:

In [6]:
# DataFrame con universidades, sedes y siglas de ciudad
dfuniversidades = pd.DataFrame({'Universidad' : ['Escuela Politécnica Nacional', 
                                                 'Escuela Politécnica del Litoral', 'Universidad Central', 
                                                'Universidad de Guayaquil', 'Universidad del Azuay'],
                               'Ciudad' : ['Quito', 'Guayaquil', 'Quito', 'Guayaquil', 'Cuenca'],
                              'Sigla' : ['UIO', 'GYE', 'UIO', 'GYE', 'CUE']})
display(dfuniversidades)

# DataFrame con ciudades, sus siglas y provincias
dfprovincias = pd.DataFrame({'Ciudad' : ['Quito', 'Guayaquil', 'Cuenca'],
                             'Sigla' : ['UIO', 'GYE', 'CUE'],
                             'Provincia' : ['Pichincha', 'Guayas', 'Azuay']})
display(dfprovincias)

display(pd.merge(dfuniversidades, dfprovincias))


,Universidad,Ciudad,Sigla
0,Escuela Politécnica Nacional,Quito,UIO
1,Escuela Politécnica del Litoral,Guayaquil,GYE
2,Universidad Central,Quito,UIO
3,Universidad de Guayaquil,Guayaquil,GYE
4,Universidad del Azuay,Cuenca,CUE


,Ciudad,Sigla,Provincia
0,Quito,UIO,Pichincha
1,Guayaquil,GYE,Guayas
2,Cuenca,CUE,Azuay


,Universidad,Ciudad,Sigla,Provincia
0,Escuela Politécnica Nacional,Quito,UIO,Pichincha
1,Universidad Central,Quito,UIO,Pichincha
2,Escuela Politécnica del Litoral,Guayaquil,GYE,Guayas
3,Universidad de Guayaquil,Guayaquil,GYE,Guayas
4,Universidad del Azuay,Cuenca,CUE,Azuay


Esto puede tener resultados inesperados cuando hay columnas con el mismo nombre, pero con significados distintos en los DataFrames:

In [7]:
# DataFrame con universidades, sedes y siglas de la UNIVERSIDAD
dfuniversidades = pd.DataFrame({'Universidad' : ['Escuela Politécnica Nacional', 
                                                 'Escuela Politécnica del Litoral', 'Universidad Central', 
                                                'Universidad de Guayaquil', 'Universidad del Azuay'],
                               'Ciudad' : ['Quito', 'Guayaquil', 'Quito', 'Guayaquil', 'Cuenca'],
                              'Sigla' : ['EPN', 'ESPOL', 'UC', 'UG', 'UA']})
display(dfuniversidades)

# DataFrame con ciudades, sus siglas y provincias
dfprovincias = pd.DataFrame({'Ciudad' : ['Quito', 'Guayaquil', 'Cuenca'],
                             'Sigla' : ['UIO', 'GYE', 'CUE'],
                             'Provincia' : ['Pichincha', 'Guayas', 'Azuay']})
display(dfprovincias)

display(pd.merge(dfuniversidades, dfprovincias))


,Universidad,Ciudad,Sigla
0,Escuela Politécnica Nacional,Quito,EPN
1,Escuela Politécnica del Litoral,Guayaquil,ESPOL
2,Universidad Central,Quito,UC
3,Universidad de Guayaquil,Guayaquil,UG
4,Universidad del Azuay,Cuenca,UA


,Ciudad,Sigla,Provincia
0,Quito,UIO,Pichincha
1,Guayaquil,GYE,Guayas
2,Cuenca,CUE,Azuay


,Universidad,Ciudad,Sigla,Provincia


Se puede especificar la(s) columna(s) sobre la(s) cual(es) se desea realizar la combinación empleando el parámetro `on`:

In [8]:
display(pd.merge(dfuniversidades, dfprovincias, on='Ciudad'))

,Universidad,Ciudad,Sigla_x,Sigla_y,Provincia
0,Escuela Politécnica Nacional,Quito,EPN,UIO,Pichincha
1,Universidad Central,Quito,UC,UIO,Pichincha
2,Escuela Politécnica del Litoral,Guayaquil,ESPOL,GYE,Guayas
3,Universidad de Guayaquil,Guayaquil,UG,GYE,Guayas
4,Universidad del Azuay,Cuenca,UA,CUE,Azuay


También puede ocurrir que la columna sobre la cual se desea hacer la combinación tenga nombres distintos en los dos DataFrames. En este caso es útil usar los parámetros `left_on` y `right_on` para indicar el nombre de la columna en cada DataFrame. En general, el término **left** (izquierdo) se usa para referirse al *primer* DataFrame mientras que el término **right** (derecho) indica el *segundo* DataFrame:

In [9]:
# DataFrame con universidades, sedes y siglas de la UNIVERSIDAD
dfuniversidades = pd.DataFrame({'Universidad' : ['Escuela Politécnica Nacional', 
                                                 'Escuela Politécnica del Litoral', 'Universidad Central', 
                                                'Universidad de Guayaquil', 'Universidad del Azuay'],
                               'Sede' : ['Quito', 'Guayaquil', 'Quito', 'Guayaquil', 'Cuenca'],
                              'Sigla' : ['EPN', 'ESPOL', 'UC', 'UG', 'UA']})
display(dfuniversidades)

# DataFrame con ciudades, sus siglas y provincias
dfprovincias = pd.DataFrame({'Ciudad' : ['Quito', 'Guayaquil', 'Cuenca'],
                             'Sigla' : ['UIO', 'GYE', 'CUE'],
                             'Provincia' : ['Pichincha', 'Guayas', 'Azuay']})
display(dfprovincias)

display(pd.merge(dfuniversidades, dfprovincias, left_on='Sede', right_on='Ciudad'))


,Universidad,Sede,Sigla
0,Escuela Politécnica Nacional,Quito,EPN
1,Escuela Politécnica del Litoral,Guayaquil,ESPOL
2,Universidad Central,Quito,UC
3,Universidad de Guayaquil,Guayaquil,UG
4,Universidad del Azuay,Cuenca,UA


,Ciudad,Sigla,Provincia
0,Quito,UIO,Pichincha
1,Guayaquil,GYE,Guayas
2,Cuenca,CUE,Azuay


,Universidad,Sede,Sigla_x,Ciudad,Sigla_y,Provincia
0,Escuela Politécnica Nacional,Quito,EPN,Quito,UIO,Pichincha
1,Universidad Central,Quito,UC,Quito,UIO,Pichincha
2,Escuela Politécnica del Litoral,Guayaquil,ESPOL,Guayaquil,GYE,Guayas
3,Universidad de Guayaquil,Guayaquil,UG,Guayaquil,GYE,Guayas
4,Universidad del Azuay,Cuenca,UA,Cuenca,CUE,Azuay


A veces uno de los DataFrames (o ambos) debe combinarse usando los valores de su índice de filas, y no los valores de una columna. Esto puede conseguirse fijando el valor del parámetro `left_index` o `right_index` a `True`, según corresponda:

In [10]:
# DataFrame con universidades, sedes y siglas de la UNIVERSIDAD
dfuniversidades = pd.DataFrame({'Universidad' : ['Escuela Politécnica Nacional', 
                                                 'Escuela Politécnica del Litoral', 'Universidad Central', 
                                                'Universidad de Guayaquil', 'Universidad del Azuay'],
                               'Sede' : ['Quito', 'Guayaquil', 'Quito', 'Guayaquil', 'Cuenca'],
                              'Sigla' : ['EPN', 'ESPOL', 'UC', 'UG', 'UA']})
display(dfuniversidades)

# DataFrame con ciudades, sus siglas y provincias
dfprovincias = pd.DataFrame({'Sigla' : ['UIO', 'GYE', 'CUE'],
                             'Provincia' : ['Pichincha', 'Guayas', 'Azuay']},
                             index = ['Quito', 'Guayaquil', 'Cuenca'])
display(dfprovincias)

display(pd.merge(dfuniversidades, dfprovincias, left_on='Sede', right_index=True))


,Universidad,Sede,Sigla
0,Escuela Politécnica Nacional,Quito,EPN
1,Escuela Politécnica del Litoral,Guayaquil,ESPOL
2,Universidad Central,Quito,UC
3,Universidad de Guayaquil,Guayaquil,UG
4,Universidad del Azuay,Cuenca,UA


,Sigla,Provincia
Quito,UIO,Pichincha
Guayaquil,GYE,Guayas
Cuenca,CUE,Azuay


,Universidad,Sede,Sigla_x,Sigla_y,Provincia
0,Escuela Politécnica Nacional,Quito,EPN,UIO,Pichincha
2,Universidad Central,Quito,UC,UIO,Pichincha
1,Escuela Politécnica del Litoral,Guayaquil,ESPOL,GYE,Guayas
3,Universidad de Guayaquil,Guayaquil,UG,GYE,Guayas
4,Universidad del Azuay,Cuenca,UA,CUE,Azuay


### Tipo de aritmética de conjuntos para las combinaciones

Suponer que se combinan dos DataFrames `df1`y `df2` usando una columna `C`. En los ejemplos anteriores, el conjunto de valores que aparecen en `C` es el mismo en ambos DataFrames. Esto no necesariamente ocurre en un caso general, y entonces es importante determinar el *tipo de aritmética de conjuntos* a utilizar en la combinación.

Por defecto, la función `merge` seleccionará aquellas filas de `df1` y `df2` cuyos valores en la columna `C` pertenezcan a la **intersección** de los conjuntos de valores de `C` en `df1` y `df2`. Este tipo de combinación se conoce como combinación interna o `inner join`.

In [11]:
# DataFrame con universidades, sedes y siglas de la UNIVERSIDAD
dfuniversidades = pd.DataFrame({'Universidad' : ['Escuela Politécnica Nacional', 
                                                 'Escuela Politécnica del Litoral', 'Universidad Central', 
                                                'Universidad de Guayaquil', 'Universidad del Azuay', 
                                                 'Universidad Técnica de Machala'],
                               'Sede' : ['Quito', 'Guayaquil', 'Quito', 'Guayaquil', 'Cuenca', 'Machala'],
                                
                              'Sigla' : ['EPN', 'ESPOL', 'UC', 'UG', 'UA', 'UTM']})
display(dfuniversidades)

# DataFrame con ciudades, sus siglas y provincias
dfprovincias = pd.DataFrame({'Sigla' : ['UIO', 'GYE', 'CUE', 'IBA'],
                             'Provincia' : ['Pichincha', 'Guayas', 'Azuay', 'Imbabura']},
                             index = ['Quito', 'Guayaquil', 'Cuenca', 'Ibarra'])
display(dfprovincias)

display(pd.merge(dfuniversidades, dfprovincias, left_on='Sede', right_index=True))


,Universidad,Sede,Sigla
0,Escuela Politécnica Nacional,Quito,EPN
1,Escuela Politécnica del Litoral,Guayaquil,ESPOL
2,Universidad Central,Quito,UC
3,Universidad de Guayaquil,Guayaquil,UG
4,Universidad del Azuay,Cuenca,UA
5,Universidad Técnica de Machala,Machala,UTM


,Sigla,Provincia
Quito,UIO,Pichincha
Guayaquil,GYE,Guayas
Cuenca,CUE,Azuay
Ibarra,IBA,Imbabura


,Universidad,Sede,Sigla_x,Sigla_y,Provincia
0,Escuela Politécnica Nacional,Quito,EPN,UIO,Pichincha
2,Universidad Central,Quito,UC,UIO,Pichincha
1,Escuela Politécnica del Litoral,Guayaquil,ESPOL,GYE,Guayas
3,Universidad de Guayaquil,Guayaquil,UG,GYE,Guayas
4,Universidad del Azuay,Cuenca,UA,CUE,Azuay


El tipo de aritmética de conjuntos a utilizar en la combinación se especifica en el parámetro `how`. Por defecto, este parámetro toma el valor de `inner`.

La combinación externa o `outer join` incluye todas las filas de `df1` y `df2` cuyos valores en la columna `C` pertenezcan a la **unión** de los conjuntos de valores de `C` en `df1` y `df2`. Esta combinación se consigue especificando el valor de `outer` para el parámetro `how`:

In [12]:
display(pd.merge(dfuniversidades, dfprovincias, left_on='Sede', right_index=True, how='outer'))

,Universidad,Sede,Sigla_x,Sigla_y,Provincia
0.0,Escuela Politécnica Nacional,Quito,EPN,UIO,Pichincha
2.0,Universidad Central,Quito,UC,UIO,Pichincha
1.0,Escuela Politécnica del Litoral,Guayaquil,ESPOL,GYE,Guayas
3.0,Universidad de Guayaquil,Guayaquil,UG,GYE,Guayas
4.0,Universidad del Azuay,Cuenca,UA,CUE,Azuay
5.0,Universidad Técnica de Machala,Machala,UTM,NaN,NaN
NaN,NaN,Ibarra,NaN,IBA,Imbabura


Notar que en una combinación externa pueden existir filas que no tienen valores para algunas columnas de `df1` o `df2`. Estos valores faltantes se completan automáticamente con `NaN`.

Un tercer tipo de aritmética de conjuntos está dado en la combinación izquierda o `left join`. En esta se incluyen *todas* las filas del primer DataFrame `df1`, y aquellas filas de `df2` cuyos valores en la columna `C` pertenezcan a la intersección de los conjuntos de valores de `C` en `df1` y `df2`. Para seleccionar este tipo, debe asignarse el valor `left` al parámetro `how`:

In [13]:
display(pd.merge(dfuniversidades, dfprovincias, left_on='Sede', right_index=True, how='left'))

,Universidad,Sede,Sigla_x,Sigla_y,Provincia
0,Escuela Politécnica Nacional,Quito,EPN,UIO,Pichincha
1,Escuela Politécnica del Litoral,Guayaquil,ESPOL,GYE,Guayas
2,Universidad Central,Quito,UC,UIO,Pichincha
3,Universidad de Guayaquil,Guayaquil,UG,GYE,Guayas
4,Universidad del Azuay,Cuenca,UA,CUE,Azuay
5,Universidad Técnica de Machala,Machala,UTM,NaN,NaN


Finalmente, el último tipo de aritmética de conjuntos se presenta en la combinación derecha o `right join`. Como se esperaría, en esta combinación se incluyen *todas* las filas del segundo DataFrame `df2`, y aquellas filas de `df1` cuyos valores en la columna `C` pertenezcan a la intersección de los conjuntos de valores de `C` en `df1` y `df2`. Para seleccionar este tipo, debe asignarse el valor `right` al parámetro `how`:

In [14]:
display(pd.merge(dfuniversidades, dfprovincias, left_on='Sede', right_index=True, how='right'))

,Universidad,Sede,Sigla_x,Sigla_y,Provincia
0.0,Escuela Politécnica Nacional,Quito,EPN,UIO,Pichincha
2.0,Universidad Central,Quito,UC,UIO,Pichincha
1.0,Escuela Politécnica del Litoral,Guayaquil,ESPOL,GYE,Guayas
3.0,Universidad de Guayaquil,Guayaquil,UG,GYE,Guayas
4.0,Universidad del Azuay,Cuenca,UA,CUE,Azuay
NaN,NaN,Ibarra,NaN,IBA,Imbabura


### Sufijos para nombres de columnas repetidos

Cuando en los dos DataFrames a combinar existen columnas con el mismo nombre, pero que no se emplean en la combinación (generalmente porque tienen significados distintos), el DataFrame combinado tendría dos o más columnas con nombres repetidos. Para evitar esto, se agregan sufijos `_x` a los nombres de las columnas del primer DataFrame y sufijos `_y` a los nombres de las columnas del segundo DataFrame. En nuestro ejemplo esto ocurre en la columna `Sigla`.

Los sufijos pueden personalizarse asignando al parámetro `suffixes` una lista con dos cadenas de caracteres, que corresponden a los sufijos a utilizar para las columnas de cada uno de los DataFrames:

In [15]:
display(pd.merge(dfuniversidades, dfprovincias, left_on='Sede', right_index=True, suffixes=['_uni', '_ciu']))

,Universidad,Sede,Sigla_uni,Sigla_ciu,Provincia
0,Escuela Politécnica Nacional,Quito,EPN,UIO,Pichincha
2,Universidad Central,Quito,UC,UIO,Pichincha
1,Escuela Politécnica del Litoral,Guayaquil,ESPOL,GYE,Guayas
3,Universidad de Guayaquil,Guayaquil,UG,GYE,Guayas
4,Universidad del Azuay,Cuenca,UA,CUE,Azuay


### El método `join`

De manera alternativa al uso de la función `merge`, dos DataFrames pueden combinarse llamando al método `join` del primero de ellos:

In [16]:
display(dfuniversidades.join(dfprovincias, on='Sede', lsuffix='_uni', rsuffix='_ciu'))

,Universidad,Sede,Sigla_uni,Sigla_ciu,Provincia
0,Escuela Politécnica Nacional,Quito,EPN,UIO,Pichincha
1,Escuela Politécnica del Litoral,Guayaquil,ESPOL,GYE,Guayas
2,Universidad Central,Quito,UC,UIO,Pichincha
3,Universidad de Guayaquil,Guayaquil,UG,GYE,Guayas
4,Universidad del Azuay,Cuenca,UA,CUE,Azuay
5,Universidad Técnica de Machala,Machala,UTM,NaN,NaN


Por defecto, `join` combina los DataFrames empleando sus índices y realiza una combinación izquierda. El tipo de aritmética de conjuntos puede seleccionarse con el parámetro `how`:

In [17]:
display(dfuniversidades.join(dfprovincias, on='Sede', lsuffix='_uni', rsuffix='_ciu', how='inner'))

,Universidad,Sede,Sigla_uni,Sigla_ciu,Provincia
0,Escuela Politécnica Nacional,Quito,EPN,UIO,Pichincha
2,Universidad Central,Quito,UC,UIO,Pichincha
1,Escuela Politécnica del Litoral,Guayaquil,ESPOL,GYE,Guayas
3,Universidad de Guayaquil,Guayaquil,UG,GYE,Guayas
4,Universidad del Azuay,Cuenca,UA,CUE,Azuay


Más información sobre la combinación de DataFrames está disponible en la documentación del sitio web de `pandas`: <https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html>.